# Results Generation

This notebook processes the results of the 2016 model from [How inter-state amity and animosity complement migration networks to drive refugee flows: A multi-layer network analysis, 1991–2016](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0245712) into predicted refugee shares for Ukraine.

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('Data_5_UPDATED.csv')\
    [['ccode1','ccode2','state.destination.name','state.origin.name']]

In [3]:
ccode_lookup = dict(zip(df.ccode1,df['state.origin.name']))

ccode_lookup_str = {}
for kk, vv in ccode_lookup.items():
    ccode_lookup_str[str(kk)] = vv

Down is ccode1, across ccode2

In [4]:
ref_preds = pd.read_csv('ref_flow_preds_2016.csv')
img_preds = pd.read_csv('immig_flow_preds_2017.csv')
img_preds_factors = pd.read_csv('immig_flow_preds_factors_2017.csv')

In [5]:
def proc_matrix(df):
    df.rename(columns={'Unnamed: 0': 'ccode1'}, inplace=True)
    df['origin'] = df.ccode1.apply(lambda x: ccode_lookup[x])
    df.rename(columns=ccode_lookup_str, inplace=True)    
    df.drop('ccode1', axis=1, inplace=True)
    df = df.set_index('origin')    
    return df

In [6]:
ref_preds = proc_matrix(ref_preds)
img_preds = proc_matrix(img_preds)
img_preds_factors = proc_matrix(img_preds_factors)

In [7]:
ref_res = pd.DataFrame(ref_preds[['Hungary','Poland','Moldova','Romania','Slovakia','Belarus','Russia']].loc['Ukraine'].sort_values(ascending=False)\
    .round(0).astype(int))
ref_res['share'] = ref_res['Ukraine'] / ref_res['Ukraine'].sum()

img_res = pd.DataFrame(img_preds[['Hungary','Poland','Moldova','Romania','Slovakia', 'Russia','Belarus']].loc['Ukraine'].sort_values(ascending=False)\
    .round(0).astype(int))
img_res['share'] = img_res['Ukraine'] / img_res['Ukraine'].sum()

img_res_factors = pd.DataFrame(img_preds_factors[['Hungary','Poland','Moldova','Romania','Slovakia', 'Russia','Belarus']].loc['Ukraine'].sort_values(ascending=False)\
    .round(0).astype(int))
img_res_factors['share'] = img_res_factors['Ukraine'] / img_res_factors['Ukraine'].sum()

In [8]:
print("Predictions from refugee model")
display(ref_res)
print()
print()
print("Predictions from immigration model")
display(img_res)
print()
print()
print("Predictions from immigration model with factors for 2010 immigration (none, low, high)")
display(img_res_factors)

Predictions from refugee model


,Ukraine,share
Russia,26807,0.878572
Belarus,801,0.026252
Hungary,711,0.023302
Romania,663,0.021729
Slovakia,627,0.020549
Moldova,495,0.016223
Poland,408,0.013372




Predictions from immigration model


,Ukraine,share
Russia,3479494,0.825381
Poland,332209,0.078804
Moldova,188926,0.044816
Belarus,152869,0.036263
Romania,24550,0.005824
Slovakia,19352,0.004591
Hungary,18221,0.004322




Predictions from immigration model with factors for 2010 immigration (none, low, high)


,Ukraine,share
Slovakia,130444372,0.273437
Poland,98593901,0.206672
Hungary,96873310,0.203065
Russia,73292076,0.153634
Romania,55640597,0.116634
Belarus,29109584,0.061019
Moldova,-6898963,-0.014462


In [9]:
ukr_results = img_res.join(ref_res, lsuffix='_immig', rsuffix='_ref')
ukr_results['predicted_shares'] = (ukr_results.share_immig + ukr_results.share_ref) / 2
ukr_results = ukr_results.reset_index().rename(columns={'index':'country'})
ukr_results = ukr_results[['country','predicted_shares']]
display(ukr_results)

,country,predicted_shares
0,Russia,0.851977
1,Poland,0.046088
2,Moldova,0.030519
3,Belarus,0.031257
4,Romania,0.013776
5,Slovakia,0.012570
6,Hungary,0.013812
